In [1]:
import altair as alt
import json
import pandas as pd

In [2]:
counties = ['B', 'K', 'Noe', 'Ooe', 'S', 'St', 'T', 'V', 'W']
counties_confirmed = ['dataTime', 'B_confirmed', 'K_confirmed', 'Noe_confirmed', 'Ooe_confirmed', 'S_confirmed', 'St_confirmed', 'T_confirmed', 'V_confirmed', 'W_confirmed']

In [3]:
df = pd.read_csv("../data/corin_data.csv")
df['dataTime'] = pd.to_datetime(df['dataTime'], format='%d.%m.%Y %H:%M:%S')
df = df[counties_confirmed]
df.tail()

,dataTime,B_confirmed,K_confirmed,Noe_confirmed,Ooe_confirmed,S_confirmed,St_confirmed,T_confirmed,V_confirmed,W_confirmed
101,2020-04-15 08:00:00,274.0,388.0,2398.0,2150.0,1176.0,1593.0,3337.0,846.0,2084.0
102,2020-04-15 15:00:00,278.0,388.0,2415.0,2162.0,1178.0,1600.0,3352.0,847.0,2101.0
103,2020-04-16 08:00:00,280.0,388.0,2427.0,2171.0,1182.0,1604.0,3362.0,851.0,2105.0
104,2020-04-16 15:00:00,286.0,389.0,2436.0,2174.0,1186.0,1617.0,3392.0,852.0,2119.0
105,2020-04-17 08:00:00,286.0,384.0,2446.0,2175.0,1189.0,1624.0,3394.0,853.0,2133.0


In [4]:
def c_line(att='total_confirmed',col='blue',scaleType='linear',label='value',highlight=False):
    the_base = alt.Chart(df, width=180, height=120).transform_filter(
        alt.datum[att] > 0  
    ).transform_timeunit(
        date='yearmonthdate(dataTime)'
    ).transform_aggregate(
        max_att='max('+att+')',
        groupby=['date']
    )
    
    the_chart = the_base.mark_line(
        color=col,
        size=2
    ).encode(
        alt.X('date:T', title=''),
        alt.Y('max_att:Q', scale=alt.Scale(type=scaleType, base=10), title='', axis=alt.Axis(minExtent=50))
    )

    if highlight is False:
        return the_chart

    the_tooltip = the_base.mark_circle(
        color=col,
        size=200,
        opacity=0
    ).encode(
        alt.X('date:T', title=''),
        alt.Y('max_att:Q', scale=alt.Scale(type=scaleType, base=10), title='', axis=alt.Axis(minExtent=50)),
        tooltip=[alt.Tooltip('date:T', title='Datum'), alt.Tooltip('max_att:Q', title=label)]
    )
    
    the_highlight = the_base.transform_window(
        sort=[alt.SortField("date", order="descending")], 
        rank="rank(date)"
    ).transform_filter(
        alt.datum.rank == 1
    )
    
    the_text = the_highlight.mark_text(
        align='left',
        color=col,
        dx=5,
        width=500
    ).encode(
        alt.X('date:T'),
        alt.Y('max_att:Q', scale=alt.Scale(type=scaleType, base=10), title=''),
        text=alt.Text("max_att:Q")
    )
    
    # transparent text so every small multiple has the same right padding
    the_alibi_text = the_highlight.mark_text(
        align='left',
        color=col,
        dx=5,
        opacity=0
    ).encode(
        alt.X('date:T'),
        alt.Y('max_att:Q', scale=alt.Scale(type=scaleType, base=10), title=''),
        text=alt.value('100000')
    )
    
    the_mark = the_highlight.mark_circle(
        color=col,
        opacity=1
    ).encode(
        alt.X('date:T'),
        alt.Y('max_att:Q', scale=alt.Scale(type=scaleType, base=10), title=''),
    )
    
    return the_chart + the_alibi_text + the_text + the_mark + the_tooltip

# ['B', 'K', 'Noe', 'Ooe', 'S', 'St', 'T', 'V', 'W']
def county_sm(highlight='T',title='Tirol'):
    highlight_color='#1f77b4'
    blur='#ddd'
    c_B = c_line('B_confirmed', highlight_color if highlight=='B' else blur, 'linear', 'Burgenland')
    c_K = c_line('K_confirmed', highlight_color if highlight=='K' else blur, 'linear', 'Kärnten')
    c_Noe = c_line('Noe_confirmed', highlight_color if highlight=='Noe' else blur, 'linear', 'Niederösterreich')
    c_Ooe = c_line('Ooe_confirmed', highlight_color if highlight=='Ooe' else blur, 'linear', 'Oberösterreich')
    c_S = c_line('S_confirmed', highlight_color if highlight=='S' else blur, 'linear', 'Salzburg')
    c_St = c_line('St_confirmed', highlight_color if highlight=='St' else blur, 'linear', 'Steiermark')
    c_T = c_line('T_confirmed', highlight_color if highlight=='T' else blur, 'linear', 'Tirol')
    c_V = c_line('V_confirmed', highlight_color if highlight=='V' else blur, 'linear', 'Vorarlberg')
    c_W = c_line('W_confirmed', highlight_color if highlight=='W' else blur, 'linear', 'Wien')
    c_highlight = c_line(highlight+'_confirmed', highlight_color, 'linear', title, True)

    return alt.layer(c_B, c_K, c_Noe, c_Ooe, c_S, c_St, c_T, c_V, c_W, c_highlight).properties(title=title)

chart_1 = (
    (county_sm('B', 'Burgenland') |
    county_sm('K', 'Kärnten') |
    county_sm('Noe', 'Niederösterreich')) &
    (county_sm('Ooe', 'Oberösterreich') |
    county_sm('S', 'Salzburg') |
    county_sm('St', 'Steiermark')) &
    (county_sm('T', 'Tirol') |
    county_sm('V', 'Vorarlberg') |
    county_sm('W', 'Wien'))
).configure_axis(
    grid=True,
    gridColor="#eee",
    domainColor="#ddd",
    tickColor="#ddd",
    labelColor="gray",
    labelBound=True,
).configure_view(
    strokeWidth=0
).configure_title(
    fontSize=12,
    fontWeight='normal',
#     anchor='start',
    color='gray'
)
    
chart_1

alt.VConcatChart(...)

In [5]:
# save chart_counties_confirmed as formatted JSON
with open('../data/chart_counties_confirmed.json', 'w') as jsonFile:
    json.dump(json.loads(chart_1.to_json()), jsonFile, indent=4, sort_keys=True)